# Profitable App Profiles for the App Store and Google Play Markets
For this project, we'll pretend we're working as data analysts for a company that builds Android and iOS mobile apps. 

We make our apps available on Google Play and the App Store for free and profit by in-app ads which is mostly influenced by the **number of users** who use the app.

Our goal for this project is to analyze data to help our developers understand what **type** of apps are likely to attract more users.

Data source:
1. [Google Play Store Apps](https://www.kaggle.com/lava18/google-play-store-apps)
2. [Mobile Apple Store Apps](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps)

In [1]:
#提取data
def extract(data_file):
    opened_data = open(data_file)
    from csv import reader
    readed_data = reader(opened_data)
    data = list(readed_data)
    return data

apple_data = extract('AppleStore.csv')
google_data = extract('googleplaystore.csv')

In [2]:
#检视data
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') 

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))
        print('\n') 
        print('------------------------------------------') 
        print('\n') 
        
explore_data(apple_data,0,4,True)
explore_data(google_data,0,4,True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7198
Number of columns: 16


------------------------------------------


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000

## appleStore.csv (7200 apps)

1. "id" : App ID
2. "track_name": App Name
3. "size_bytes": Size (in Bytes)
4. "currency": Currency Type
5. "price": Price amount
6. "ratingcounttot": User Rating counts (for all version)
7. "ratingcountver": User Rating counts (for current version)
8. "user_rating" : Average User Rating value (for all version)
9. "userratingver": Average User Rating value (for current version)
10. "ver" : Latest version code
11. "cont_rating": Content Rating
12. "prime_genre": Primary Genre
13. "sup_devices.num": Number of supporting devices
14. "ipadSc_urls.num": Number of screenshots showed for display
15. "lang.num": Number of supported languages
16. "vpp_lic": Vpp Device Based Licensing Enabled

## Define a function that can find missing value and remove the data:

In [3]:
def findmiss(dataset):
    header = dataset[0]
    for row in dataset:
        if len(row) != len(header):
            print(header)
            print(row)
            print('miss position:',  dataset.index(row))
            print(len(row))
            print('missing data in',dataset.index(row),'has been deleted')
            del dataset[dataset.index(row)]

In [4]:
findmiss(apple_data)

In [5]:
findmiss(google_data)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
miss position: 10473
12
missing data in 10473 has been deleted


## Two way to find duplications

- way 1:

In [6]:
duplications = []
for row1 in google_data[1:]:
    name = row1[0]
    for row2 in google_data[google_data.index(row1)+1:]:
        if name == row2[0] and name not in duplications:
            duplications.append(name)

print(len(duplications))

798


- way 2:

In [7]:
duplicate_apps = []
unique_apps = []
    
for app in google_data[1:]:
    name = app[0]
    if name in unique_apps and name not in duplicate_apps:
            duplicate_apps.append(name)
    else:
        unique_apps.append(name)
            
print(len(duplicate_apps))



798


Results above denote there are 798 apps that appear two times or more in our data.

---

In [8]:
for app in google_data:
    name = app[0]
    if name == 'Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


From result above, we can see the only difference between duplications is the **fourth** feature, that is, the number of reviews. We can use this information to build a criterion for removing the duplicates. **The higher** the number of reviews, **the more recent** the data should be. Wiht this thought, we will only keep the very **latest** observation of each app in our data.

- Step 1

Create a dictionary where each key is a unique app name and the corresponding dictionary value is the highest number of reviews of that app.

In [9]:
reviews_max = {}

for row in google_data[1:]:
    name = row[0]
    n_reviews = float(row[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
len(reviews_max)

9659

- Step 2

Use the dictionary to remove the duplicate rows:


In [10]:
android_clean = []

for row in google_data[1:]:
    name = row[0]
    n_reviews = float(row[3])
    
    if (n_reviews == reviews_max[name]):
        android_clean.append(row)

print(android_clean[:4])
len(android_clean)

[['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'], ['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'], ['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up'], ['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']]


10054

In [11]:
duplications2 = []
for row1 in android_clean:
    name = row1[0]
    for row2 in android_clean[android_clean.index(row1)+1:]:
        if name == row2[0] and name not in duplications2:
            duplications2.append(name)

print(len(duplications2))
duplications2[:3]

336


['Ebook Reader', 'Docs To Go™ Free Office Suite', 'Google My Business']

In [12]:
for row in android_clean:
    name = row[0]
    if name == 'Insightly CRM':
        print(row)

['Insightly CRM', 'BUSINESS', '3.8', '1383', '51M', '100,000+', 'Free', '0', 'Everyone', 'Business', 'July 12, 2018', '3.24.1', '5.0 and up']
['Insightly CRM', 'BUSINESS', '3.8', '1383', '51M', '100,000+', 'Free', '0', 'Everyone', 'Business', 'July 12, 2018', '3.24.1', '5.0 and up']


从上方运行结果可以知道，**有些重复数据的`reviews_max`也是一样的**，这样通过判断最大值插入时，会把所有最大的都插入。

因此我们需要定义一个 empty list -- `already_added`

In [13]:
android_clean = []
already_added = []

for row in google_data[1:]:
    name = row[0]
    n_reviews = float(row[3])
    
    if (n_reviews == reviews_max[name]) and (name not in already_added):
        android_clean.append(row)
        already_added.append(name)

print(android_clean[:4])
len(android_clean)

[['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'], ['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'], ['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up'], ['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']]


9659

## Detect non-English apps


In [14]:
string = 'abc'
print(string[0])
print(string[1])
print(string[2])

a
b
c


基于此，我们可以遍历`app`的`name`，根据其**ASCII**值来判断是否为英文app，通常:
>The numbers corresponding to the characters we commonly use in an English text are all in the range 0 to 127, according to the ASCII (American Standard Code for Information Interchange) system. 

可以通过`ord()`函数得到相应字符的ASCII number

In [15]:
print(ord('a'))

97


In [16]:
def Eng_det(string):
    for char in string:
        if 127 < ord(char) or ord(char) <= 0:
            return False
    return True

Eng_det('Docs To Go™ Free Office Suite')
        

False

We wrote a function that detects non-English app names, but we saw that the function couldn't correctly identify certain English app names like `'Docs To Go™ Free Office Suite'`

In [17]:
print(ord('™'))
print(ord('😜'))

8482
128540


针对这一情况，我们的策略是:

> we'll only remove an app if its name has **more than three** characters with corresponding numbers falling outside the ASCII range.

In [18]:
def Eng_det(string):
    n_non_Eng = 0
    n_Eng = 0
    for char in string:
        if 127 < ord(char) or ord(char) <= 0:
            n_non_Eng += 1
        else:
            n_Eng +=1
            
    if n_non_Eng > 3:
        return False
    elif n_Eng < n_non_Eng:
        return False
    else:
        return True       

In [19]:
android_clean_Eng = []
apple_clean_Eng = []

for row in android_clean:
    name = row[0]
    if Eng_det(name):
        android_clean_Eng.append(row)

for row in apple_data[1:]:
    name = row[0]
    if Eng_det(name):
        apple_clean_Eng.append(row)


In [20]:
explore_data(android_clean_Eng,0,2,True)
explore_data(apple_clean_Eng,0,2,True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 9614
Number of columns: 13


------------------------------------------


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows: 7197
Number of columns: 16


------------------------------------------




Recall that 我们的app是基于内置广告浏览量获利的免费应用，因此我们一个着重关注**free app**.

In [21]:
apple = []
google = []
for row in android_clean_Eng:
    price = row[7]
    if price == '0.0' or price == '0':
        google.append(row)
        
for row in apple_clean_Eng:
    price = row[4]
    if price == '0.0' or price == '0':
        apple.append(row)
        
explore_data(apple,0,2,True)
explore_data(google,0,2,True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows: 4056
Number of columns: 16


------------------------------------------


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 8864
Number of columns: 13


------------------------------------------




To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we develop it further.
3. If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful on both markets. For instance, a profile that works well for both markets might be a productivity app that makes use of gamification.

In [22]:
print(google_data[0])
print(apple_data[0])

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


In [23]:
google_freq = {}

f = [[4,'a',4],[2,'b',2]]
sorted(f)

[[2, 'b', 2], [4, 'a', 4]]

In [24]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    #排序
    table_sorted = sorted(table_display, reverse = True)
    
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [25]:
def freq_table(dataset, index):
    #频数表
    freq = {}
    for row in dataset:
        var = row[index]
        if var in freq:
            freq[var] += 1
        else:
            freq[var] = 1
    
    #百分比
    total = len(dataset)
    for element in freq:
        freq[element] /= total/100
    return freq    

In [26]:
# apple, prime_genre
display_table(apple,11)

Games : 55.645956607495066
Entertainment : 8.234714003944772
Photo & Video : 4.117357001972386
Social Networking : 3.5256410256410255
Education : 3.254437869822485
Shopping : 2.983234714003945
Utilities : 2.687376725838264
Lifestyle : 2.3175542406311638
Finance : 2.0710059171597632
Sports : 1.947731755424063
Health & Fitness : 1.8737672583826428
Music : 1.6518737672583825
Book : 1.6272189349112425
Productivity : 1.5285996055226823
News : 1.4299802761341223
Travel : 1.3806706114398422
Food & Drink : 1.0601577909270217
Weather : 0.7642998027613411
Reference : 0.49309664694280075
Navigation : 0.49309664694280075
Business : 0.49309664694280075
Catalogs : 0.22189349112426035
Medical : 0.1972386587771203


---

In [27]:
# google, Genres
display_table(google,9)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.700361010830325
Medical : 3.5311371841155235
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.237815884476534
Action : 3.1024368231046933
Health & Fitness : 3.079873646209386
Photography : 2.9444945848375452
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.041967509025271
Dating : 1.861462093862816
Arcade : 1.8501805054151625
Video Players & Editors : 1.7712093862815885
Casual : 1.759927797833935
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418774
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075813

In [28]:
# google, Category
display_table(google,1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.700361010830325
MEDICAL : 3.5311371841155235
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.237815884476534
HEALTH_AND_FITNESS : 3.079873646209386
PHOTOGRAPHY : 2.9444945848375452
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768953
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418774
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075813
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 0

## Most Popular Apps by Genre on the App Store

分析什么genre的app可以获得更多的用户，那么仅看市场中各个genre的占比/频数是不够的，我们需要计算每个genre的**平均下载量/打分量**

以下显示了two ways that can do this, 方法的本质是一样的，只是写法不同。

- way 1：

In [48]:
def count_by_group(dataset,group_index,number_index):
    count = {}
    for row in dataset:
        group = row[group_index]
        if group in count:
            count[group] += float(row[number_index])
        else:
            count[group] = float(row[number_index])
    return count

In [49]:
def avg_for_group(dataset,group_index,number_index):
    count = count_by_group(dataset,group_index,number_index)
    freq = freq_table(dataset,group_index)
    
    # freq_table()返回的的值是百分数，转化为频数
    total = len(dataset)
    for element in freq:
        freq[element] = freq[element] * total / 100
    
    #计算avg
    avg = {}
    for c in count:
        for f in freq:
            if c == f:
                avg[c] = count[c] / freq[f]
    return avg

- way 2:

In [60]:
def avg_for_group2(dataset,group_index,number_index):
    freq = freq_table(dataset,group_index)
    
    # freq_table()返回的值是百分数，转化为频数
    total = len(dataset)
    for element in freq:
        freq[element] = freq[element] * total / 100
    
    #计算avg
    avg = {}
    for genre in freq:
        total = 0
        len_genre = 0
        for row in dataset:
            genre_app = row[group_index]
            if genre_app == genre:
                total += float(row[number_index])
                len_genre += 1
                
        average = total / len_genre
        avg[genre] = average
                
    return avg

In [65]:
def display_avg_table(dataset,group_index,number_index,avg_func):
    table = avg_func(dataset,group_index,number_index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    #排序
    table_sorted = sorted(table_display, reverse = True)
    
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [67]:
display_avg_table(apple,11,5,avg_for_group)

Reference : 67447.90000000001
Music : 56482.02985074628
Social Networking : 53078.195804195806
Weather : 47220.93548387097
Photo & Video : 27249.892215568867
Navigation : 25972.050000000003
Travel : 20216.01785714286
Food & Drink : 20179.093023255813
Sports : 20128.974683544307
Health & Fitness : 19952.315789473687
Productivity : 19053.887096774197
Games : 18924.68896765618
Shopping : 18746.677685950413
News : 15892.724137931034
Utilities : 14010.100917431193
Finance : 13522.261904761905
Entertainment : 10822.961077844313
Lifestyle : 8978.308510638299
Book : 8498.333333333336
Business : 6367.800000000001
Education : 6266.333333333335
Catalogs : 1779.5555555555557
Medical : 459.75000000000006


In [66]:
display_avg_table(apple,11,5,avg_for_group2)

Reference : 67447.9
Music : 56482.02985074627
Social Networking : 53078.195804195806
Weather : 47220.93548387097
Photo & Video : 27249.892215568863
Navigation : 25972.05
Travel : 20216.01785714286
Food & Drink : 20179.093023255813
Sports : 20128.974683544304
Health & Fitness : 19952.315789473683
Productivity : 19053.887096774193
Games : 18924.68896765618
Shopping : 18746.677685950413
News : 15892.724137931034
Utilities : 14010.100917431193
Finance : 13522.261904761905
Entertainment : 10822.961077844311
Lifestyle : 8978.308510638299
Book : 8498.333333333334
Business : 6367.8
Education : 6266.333333333333
Catalogs : 1779.5555555555557
Medical : 459.75


## Most Popular Apps by Genre on Google Play

In [68]:
display_table(google,5)

1,000,000+ : 15.72653429602888
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.1985559566787
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.772111913357401
5,000+ : 4.512635379061372
10+ : 3.542418772563177
500+ : 3.2490974729241877
50,000,000+ : 2.3014440433212995
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.7897111913357401
1+ : 0.5076714801444043
500,000,000+ : 0.27075812274368233
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


To remove `'+'` and `','` from the string, we can use `str.replace(old, new)` method.

In [78]:
def avg_google(dataset,group_index,number_index):
    freq = freq_table(dataset,group_index)
    
    # freq_table()返回的值是百分数，转化为频数
    total = len(dataset)
    for element in freq:
        freq[element] = freq[element] * total / 100
    
    #计算avg
    avg = {}
    for genre in freq:
        total = 0
        len_genre = 0
        for row in dataset:
            genre_app = row[group_index]
            if genre_app == genre:
                n = row[number_index]
                n = n.replace('+','')
                n = n.replace(',','')
                total += float(n)
                len_genre += 1
                
        average = total / len_genre
        avg[genre] = average
                
    return avg

In [79]:
display_avg_table(google,1,5,avg_google)

COMMUNICATION : 38456119.167247385
VIDEO_PLAYERS : 24727872.452830188
SOCIAL : 23253652.127118643
PHOTOGRAPHY : 17840110.40229885
PRODUCTIVITY : 16787331.344927534
GAME : 15588015.603248259
TRAVEL_AND_LOCAL : 13984077.710144928
ENTERTAINMENT : 11640705.88235294
TOOLS : 10801391.298666667
NEWS_AND_MAGAZINES : 9549178.467741935
BOOKS_AND_REFERENCE : 8767811.894736841
SHOPPING : 7036877.311557789
PERSONALIZATION : 5201482.6122448975
WEATHER : 5074486.197183099
HEALTH_AND_FITNESS : 4188821.9853479853
MAPS_AND_NAVIGATION : 4056941.7741935486
FAMILY : 3695641.8198090694
SPORTS : 3638640.1428571427
ART_AND_DESIGN : 1986335.0877192982
FOOD_AND_DRINK : 1924897.7363636363
EDUCATION : 1833495.145631068
BUSINESS : 1712290.1474201474
LIFESTYLE : 1437816.2687861272
FINANCE : 1387692.475609756
HOUSE_AND_HOME : 1331540.5616438356
DATING : 854028.8303030303
COMICS : 817657.2727272727
AUTO_AND_VEHICLES : 647317.8170731707
LIBRARIES_AND_DEMO : 638503.734939759
PARENTING : 542603.6206896552
BEAUTY : 51315